## Back End in one
这个仅仅是本地测试：拿到音频-> 提取信息 -> 在数据库里找数据 -> 给客户返回数据（前端）-> 上传Azure 数据库

In [1]:
# 接收前端音频
# Connection part
from fastapi import FastAPI, File, UploadFile

app = FastAPI()

@app.post("/upload_audio")
async def upload_audio(file: UploadFile = File(...)):
    with open("test1.wav", "wb") as f:
        f.write(await file.read())
    return {"msg": "Audio uploaded successfully"}


In [2]:
# Part 1: Audio to Text
# 这里需要接收前端音频 来load他们接受的音频转文本
# 用的Whisper
import whisper

model = whisper.load_model("base")

def speech_to_text(audio_path):
    result = model.transcribe(audio_path, language='en')  # or 'zh' for Chinese
    return result['text']
test1 = speech_to_text(
    'test2.wav'
)
print(test1)

/opt/homebrew/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hi my name is Harry my phone number is 0434 322 781 I want to inspect a property at Flinders Street Melbourne thank you


In [3]:
# 用Azure open AI做关键字提取，现在resource里创建一个ai，然后在azure portal里面为ai选择 engine
# Part 2:Extract the information from the text
import openai
import json

openai.api_type = "azure"
openai.api_key = "mSSedelOpALeOCyKP4ssEipRpAkgAZz3v1kTIHBSGnrJqprIo349JQQJ99BGACL93NaXJ3w3AAABACOGf4ui" # API Key
openai.api_base = "https://aivoicetest.openai.azure.com/" # API Endpoint
openai.api_version = "2024-02-15-preview"   # 

# Train Azure AI and letting it extract the information
def extract_info(text):
    prompt = (
    "Extract the client's name, phone number, and property address from the text below. "
    "Return ONLY a JSON object (no extra text, no explanations, no code block).\n"
    f"Text: {text}\n"
    "Example output: {\"name\": \"Kevin Su\", \"phone\": \"0402662860\", \"address\": \"101/25-33 Wills Street Melbourne VIC 3000\"}"
)

    resp = openai.ChatCompletion.create(
        engine= "gpt4o_voicebot",   
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    try:
        info = json.loads(resp.choices[0].message['content'])
    except:
        info = {}
    return info

test2 = extract_info(test1)
address = test2['address']
print(test1)
print(test2)


# Success

 Hi my name is Harry my phone number is 0434 322 781 I want to inspect a property at Flinders Street Melbourne thank you
{'name': 'Harry', 'phone': '0434 322 781', 'address': 'Flinders Street Melbourne'}


In [4]:
# 在数据库里面找到符合条件的房源然后上传
import pyodbc

def query_property_azure(address):
    conn = pyodbc.connect(
        "DRIVER={ODBC Driver 18 for SQL Server};"
        "SERVER=voiceai-sql-server.database.windows.net;"
        "DATABASE=RealEstateConvoAI;"
        "UID=lorraine;"
        "PWD=Test123!Temp;"
        "Encrypt=yes;"
        "TrustServerCertificate=no;"
        "Connection Timeout=30;"
    )
    cursor = conn.cursor()
# check and find whether you have any matched data in the data base
    cursor.execute("SELECT * FROM [House_Data] WHERE [name] LIKE ?", ('%' + address + '%',))
    result = cursor.fetchone()
    return result if result else "Not found"

# address = "Spencer Street Melbourne"   用上面的address
result = query_property_azure(address)
print(result)
# Success

('1106/555 Flinders Street Melbourne VIC 3000', 'Apartment / Unit / Flat', 2023, '[-37.8209711, 144.9564827]', 3000, None, 'Melbourne', 1.0, 1.0, None, 129324.0, 47744.20000000007, 1.4, 3.2, 1.1, 2.9, '370.0', 4.0, 3000.0, "['Melbourne']")


## 这里有一个问题就是not found后面要说其他反馈的话

In [5]:
def property_summary(row):
    if not row:
        return "Sorry, no matching property found."
    return (
        f"Below is the best matched property for your search:\n"
        f"Address: {row[0]}\n"
        f"Type: {row[1]}\n"
        f"Year built: {row[2]}, Region: {row[18]}\n"
        f"Rooms: {row[7]} bed(s), {row[8]} bath(s), {row[9]} car space(s)\n"
        f"Weekly rent: ${row[16]}\n"
        f"Located near {row[6]}, Postcode: {row[4]}.\n"
        f"For more details, please contact us!"
    )
response = property_summary(result)
print(response)

Below is the best matched property for your search:
Address: 1106/555 Flinders Street Melbourne VIC 3000
Type: Apartment / Unit / Flat
Year built: 2023, Region: 3000.0
Rooms: 1.0 bed(s), 1.0 bath(s), None car space(s)
Weekly rent: $370.0
Located near Melbourne, Postcode: 3000.
For more details, please contact us!


In [9]:
# Part 4: Convert the information into audio
# This part need to connect to the front end
# 这个应该是把提取到的房地产信息转化成音频 音频的名字叫做out_file
from gtts import gTTS

def text_to_speech(text, out_file="result.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(out_file)
    return out_file

text_to_speech(response, "result.mp3")
# Success


'result.mp3'

In [7]:
# Updating the information into Azure SQL
import pyodbc
# 这里的data应该是test2 里面的内容 
def upload_to_azure_sql(data):
    conn = pyodbc.connect(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=voiceai-sql-server.database.windows.net;"
    "DATABASE=RealEstateConvoAI;"
    "UID=lorraine;"
    "PWD=Test123!Temp;"
)
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO Customer_Table (name, phone, address) VALUES (?, ?, ?)",
        data["name"], data["phone"], data["address"]
    )
    conn.commit()
    conn.close()
upload_to_azure_sql(test2)

## 现阶段已经完成本地测试了 需要做的就是END TO END连接；
比如说流程上的优化 Voice cancellation这种
采集客户的各类信息 匹配的房源 这类